<a href="https://colab.research.google.com/github/armelida/MELIDA/blob/main/notebooks/prompting_strategy_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("MELIDA: Prompting Strategy Evaluator is ready!")


MELIDA: Prompting Strategy Evaluator is ready!
